In [2]:
import shutil
from requests import get

from google.colab import drive, files
# mount Google Drive


Mounted at /content/drive


In [1]:
!nvidia-smi

Wed Mar  8 13:10:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    31W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import re
import os
import pandas as pd
import numpy as np
import random
import joblib
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold

import warnings
warnings.simplefilter("ignore")

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from pymatgen.core import Structure

In [8]:
class CFG:
  base_dir = r"C:\Users\NDO00\OneDrive\デスクトップ\MI\mi_book\Geo_CGCNN"
  data_dir = os.path.join(base_dir, "nishika_csv")
  output_dir_cif = os.path.join(base_dir, "database/cif")
  output_dir_target = os.path.join(base_dir, "database")
  sub_dir = os.path.join(base_dir, "data")
  target_col = "formation_energy_per_atom"

## データの形式確認

In [9]:
train_df = pd.read_csv(f"{CFG.data_dir}/train.csv",na_values=[""],
                       keep_default_na=False,dtype={"formula_pretty":str})
train_df.head()

,id,Ac,Ag,Al,Ar,As,Au,B,Ba,Be,...,Xe,Y,Yb,Zn,Zr,nelements,nsites,formula_pretty,cif,formation_energy_per_atom
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,19,K2MgMo2(H2O5)2,# generated using pymatgen\r\ndata_K2MgMo2(H2O...,-1.996778
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,48,CaGd4O7,# generated using pymatgen\r\ndata_CaGd4O7\r\n...,-3.754292
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,6,Sb2Ru,# generated using pymatgen\r\ndata_Sb2Ru\r\n_s...,-0.381808
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,4,CaGdCd2,# generated using pymatgen\r\ndata_CaGdCd2\r\n...,-0.345960
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,16,Mg14CrW,# generated using pymatgen\r\ndata_Mg14CrW\r\n...,0.204384


In [10]:
train_df.tail()

,id,Ac,Ag,Al,Ar,As,Au,B,Ba,Be,...,Xe,Y,Yb,Zn,Zr,nelements,nsites,formula_pretty,cif,formation_energy_per_atom
71724,71725,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,20,NaEr(PS3)2,# generated using pymatgen\r\ndata_NaEr(PS3)2\...,-1.205766
71725,71726,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,64,LiSbPO5,# generated using pymatgen\r\ndata_LiSbPO5\r\n...,-2.334947
71726,71727,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,4,LiSbRh2,# generated using pymatgen\r\ndata_LiSbRh2\r\n...,-0.588092
71727,71728,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4,16,Mg6SiBiO8,# generated using pymatgen\r\ndata_Mg6SiBiO8\r...,-2.264738
71728,71729,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,2,6,NaAg2,# generated using pymatgen\r\ndata_NaAg2\r\n_s...,-0.100161


In [11]:
test_df = pd.read_csv(f"{CFG.data_dir}/test.csv",na_values=[""],
                       keep_default_na=False,dtype={"formula_pretty":str})
test_df.head()

,id,Ac,Ag,Al,Ar,As,Au,B,Ba,Be,...,W,Xe,Y,Yb,Zn,Zr,nelements,nsites,formula_pretty,cif
0,71730,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,30,Ca2Ga2Si(HO4)2,# generated using pymatgen\r\ndata_Ca2Ga2Si(HO...
1,71731,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,5,Ca(GeRu)2,# generated using pymatgen\r\ndata_Ca(GeRu)2\r...
2,71732,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,4,46,Ba2LaNb5O15,# generated using pymatgen\r\ndata_Ba2LaNb5O15...
3,71733,0,0,0,0,0,0,16,0,0,...,0,0,0,0,0,0,3,24,DyB4Mo,# generated using pymatgen\r\ndata_DyB4Mo\r\n_...
4,71734,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,42,Sr2La4Mg2RuO12,# generated using pymatgen\r\ndata_Sr2La4Mg2Ru...


In [45]:
test_df.tail()

,id,Ac,Ag,Al,Ar,As,Au,B,Ba,Be,...,W,Xe,Y,Yb,Zn,Zr,nelements,nsites,formula_pretty,cif
28686,100416,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,10,La2CdRh2,# generated using pymatgen\r\ndata_La2CdRh2\r\...
28687,100417,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,64,InPO6,# generated using pymatgen\r\ndata_InPO6\r\n_s...
28688,100418,0,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,5,40,K5Na3Nb4(WO6)4,# generated using pymatgen\r\ndata_K5Na3Nb4(WO...
28689,100419,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,28,ReTeSCl4,# generated using pymatgen\r\ndata_ReTeSCl4\r\...
28690,100420,0,0,0,0,0,0,0,0,0,...,0,0,0,18,8,0,3,44,Yb9Zn4Bi9,# generated using pymatgen\r\ndata_Yb9Zn4Bi9\r...


nishikaのテーブルデータのcif列からcifファイルを作成し、フォルダに保存する。

In [12]:
#trainデータの場合
for i in range(len(train_df)):
  cif_str = train_df["cif"].values[i]
  num=train_df["id"].values[i]
  f = open(f"{CFG.output_dir_cif}/mp-{num}.cif", 'w')
  for line in cif_str.split("\n"):
    f.write(line+"\n")
    
  f.close()

In [13]:
#テストデータの場合
for i in range(len(test_df)):
  cif_str = test_df["cif"].values[i]
  num=test_df["id"].values[i]
  f = open(f"{CFG.output_dir_cif}/mp-{num}.cif", 'w')
  for line in cif_str.split("\n"):
    f.write(line+"\n")
    
  f.close()

ターゲット変数の格納されたcsvを作る(trainデータ)。

In [20]:
cols = ["id", "formation_energy_per_atom", "pretty_formula"]

# dataframeの作成
train_df_target = pd.DataFrame(columns=cols)
train_df_target["formation_energy_per_atom"]=train_df["formation_energy_per_atom"]
train_df_target["pretty_formula"]=train_df["formula_pretty"]

for i in range(len(train_df)):
  num=train_df["id"].values[i]

  train_df_target.loc[train_df_target.index[i],"id"]=f"mp-{num}"
train_df_target

,id,formation_energy_per_atom,pretty_formula
0,mp-1,-1.996778,K2MgMo2(H2O5)2
1,mp-2,-3.754292,CaGd4O7
2,mp-3,-0.381808,Sb2Ru
3,mp-4,-0.345960,CaGdCd2
4,mp-5,0.204384,Mg14CrW
...,...,...,...
71724,mp-71725,-1.205766,NaEr(PS3)2
71725,mp-71726,-2.334947,LiSbPO5
71726,mp-71727,-0.588092,LiSbRh2
71727,mp-71728,-2.264738,Mg6SiBiO8


In [ ]:
train_df_target.to_csv(f"{CFG.output_dir_target}/train_target.csv")

ターゲット変数の格納されたcsvを作る(testデータ)。

In [14]:
cols = ["id", "formation_energy_per_atom", "pretty_formula"]

# dataframeの作成
test_df_target = pd.DataFrame(columns=cols)
test_df_target["formation_energy_per_atom"]=[0.5] * len(test_df)
test_df_target["pretty_formula"]=test_df["formula_pretty"]

for i in range(len(test_df)):
  num=test_df["id"].values[i]

  test_df_target.loc[test_df_target.index[i],"id"]=f"mp-{num}"
test_df_target

,id,formation_energy_per_atom,pretty_formula
0,mp-71730,0.5,Ca2Ga2Si(HO4)2
1,mp-71731,0.5,Ca(GeRu)2
2,mp-71732,0.5,Ba2LaNb5O15
3,mp-71733,0.5,DyB4Mo
4,mp-71734,0.5,Sr2La4Mg2RuO12
...,...,...,...
28686,mp-100416,0.5,La2CdRh2
28687,mp-100417,0.5,InPO6
28688,mp-100418,0.5,K5Na3Nb4(WO6)4
28689,mp-100419,0.5,ReTeSCl4


In [ ]:
test_df_target.to_csv(f"{CFG.output_dir_target}/test_target.csv")

ターゲット変数の結合

In [22]:
df_target=pd.concat([train_df_target,test_df_target],ignore_index=True)
df_target

,id,formation_energy_per_atom,pretty_formula
0,mp-1,-1.996778,K2MgMo2(H2O5)2
1,mp-2,-3.754292,CaGd4O7
2,mp-3,-0.381808,Sb2Ru
3,mp-4,-0.345960,CaGdCd2
4,mp-5,0.204384,Mg14CrW
...,...,...,...
100415,mp-100416,0.500000,La2CdRh2
100416,mp-100417,0.500000,InPO6
100417,mp-100418,0.500000,K5Na3Nb4(WO6)4
100418,mp-100419,0.500000,ReTeSCl4


In [23]:
df_target.to_csv(f"{CFG.output_dir_target}/targets_MP_Ef.csv")